In [1]:
import duckdb
import pandas as pd

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

Deploy FastAPI apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


In [5]:
%%sql 
INSTALL spatial;
LOAD spatial;
INSTALL h3 FROM community;
LOAD h3;
INSTALL postgres;
LOAD postgres;


Running query in 'duckdb'

Success


In [3]:
%sql CREATE TABLE source_fua AS SELECT * FROM ST_Read('../_source/fua.gpkg');

Running query in 'duckdb'

Count
9031


In [36]:
%%sql
CREATE OR REPLACE TABLE cities AS
SELECT id as efua_id, name, fua_pop, cnt_iso FROM postgres_scan('host=localhost port=5432 user=jonathan dbname=territoires.fyi', 'dbt_urbanized_fyi', 'cities');

Running query in 'duckdb'

Count
2000


In [18]:
%sql SELECT * FROM cities LIMIT 5;

Running query in 'duckdb'

efua_id,name,fua_pop,cnt_iso
9725,Guangzhou,45640589,['CHN']
4900,Jakarta,39808169,['IDN']
5129,Tokyo,36471787,['JPN']
7466,Delhi [New Delhi],30076708,['IND']
10097,Shanghai,26920218,['CHN']


In [19]:
%%sql
CREATE OR REPLACE TABLE 
h3_efua_pop AS
SELECT * FROM postgres_scan('host=localhost port=5432 user=jonathan dbname=territoires.fyi', 'dbt_urbanized_fyi', 'efua_h3_pop');

Running query in 'duckdb'

Count
3601676


In [23]:
%%sql
CREATE OR REPLACE TABLE 
source_kontur AS
SELECT h3, population FROM ST_Read('../_source/kontur.gpkg');


Running query in 'duckdb'

Count
32957699


In [27]:
%%sql
CREATE OR REPLACE TABLE h3_efua_pop AS
SELECT h3_efua_pop.h3, h3_efua_pop.efua_id, source_kontur.population 
FROM h3_efua_pop 
JOIN source_kontur ON h3_efua_pop.h3 = source_kontur.h3;

Running query in 'duckdb'

Count
3086206


In [37]:
%%sql
CREATE OR REPLACE TABLE cities AS
SELECT cities.*, kontur_population.kontur_population AS kontur_population
FROM cities
JOIN (
  SELECT cities.efua_id, SUM(h3_efua_pop.population) AS kontur_population
  FROM cities
  JOIN h3_efua_pop ON cities.efua_id = h3_efua_pop.efua_id
  GROUP BY cities.efua_id
) AS kontur_population
ON cities.efua_id = kontur_population.efua_id;

Running query in 'duckdb'

Count
2000


In [43]:
%%sql
SELECT 
  efua_id,
  name,
  cnt_iso,
  CAST(fua_pop AS INTEGER) as fua_pop,
  CAST(kontur_population AS INTEGER) as kontur_pop
FROM cities LIMIT 5;

Running query in 'duckdb'

efua_id,name,cnt_iso,fua_pop,kontur_pop
10181,Yiwu,['CHN'],1706311,1790056
10078,Hangzhou,['CHN'],9581084,8966182
9667,Ji'an,['CHN'],435431,443573
8043,Handan,['CHN'],1698252,2445124
8798,Damoh,['IND'],384318,167063


In [45]:
%%sql
COPY (
  SELECT 
    efua_id,
    name,
    cnt_iso,
    CAST(fua_pop AS INTEGER) as fua_pop,
    CAST(kontur_population AS INTEGER) as kontur_pop
  FROM cities
) TO 'cities.csv' (HEADER, DELIMITER ',');
COPY (
  SELECT
    h3,
    efua_id,
    CAST(population AS INTEGER) as population
  FROM h3_efua_pop
) TO 'cities_h3_pop.csv' (HEADER, DELIMITER ',');

Running query in 'duckdb'

Count
3086206
